The aim is to reduce the weight of the database.

In [1]:
import pandas as pd
import numpy as np
import pickle

# Utilities

In [2]:
#
#  Dataframe utilities
#

path_data = "data/"

def col_perc(df, col):
    print(str(round(df[col].nunique()/df.shape[0]*100, 2)) + "%")
    
def get_save(df_raw, df):
    print(str(round(df.memory_usage(deep = True).sum()/df_raw.memory_usage(deep = True).sum()*100,2))\
          + "% of raw !")

def get(name):
    return pd.read_csv(path_data + name + ".csv")

def save(df, name):
    path = path_data + name + ".pickle"
    return pickle.dump(df, open(path,"wb"))

    


#
#  Emails to foreign key utilities
#

emails = pd.DataFrame(columns = ["email", "emp_id"])

def add_emails(data):
    global emails
    data = np.array(data)
    k = len(data.shape)
    if k == 1:
        temp = pd.DataFrame(data, columns = ["email"])
        temp["emp_id"] = pd.Series(0, index = temp.index)
    elif k == 2:
        temp = pd.DataFrame(data, columns = emails.columns.values)
    else:
        raise ValueError("Data is not shape for emails !")
    
    new_emails = temp.loc[~temp.email.isin(emails.email)].drop_duplicates()
    emails = emails.append(new_emails, ignore_index=True)
    emails = emails.astype({"emp_id" : np.uint32})
    emails.index.name = "e_id"
    
    print(str(new_emails.shape[0]) + " emails added from " + str(temp.shape[0])\
          + " emails given ("+str(emails.shape[0])+" in memory).")
    
def update_emails(df, col):
    email_keys = dict(zip(emails.email.values, emails.index))
    df[col] = df[col].apply(lambda x : email_keys[x])
    df = df.rename(columns = {col: "e_id"})
    df = df.astype({"e_id" : np.uint32})
    return df
    
    

# Employees

In [3]:
emp_raw = get("raw_employeelist")
emp = emp_raw.rename(columns = {"eid" : "emp_id"})
emp = emp.astype({"status" : "category"})
emp = emp.set_index("emp_id")
emp = emp.drop_duplicates()

# Append mails with employee from employeelist
res = []
cols = ["Email_id", "Email2", "Email3", "EMail4"]
for index, row in emp.iterrows():
    for col in cols:
        x = row[col]
        if (x == x):
            res.append([x,index])
add_emails(res)

# Removing emails columns
emp = emp[["firstName", "lastName", "folder", "status"]]

#Save
save(emp, "employees")

#Show
get_save(emp_raw, emp)
emp.head()

297 emails added from 297 emails given (297 in memory).
31.4% of raw !


,firstName,lastName,folder,status
emp_id,,,,
13,Marie,Heard,heard-m,NaN
6,Mark,Taylor,taylor-m,Employee
19,Lindy,Donoho,donoho-l,Employee
115,Lisa,Gang,gang-l,NaN
129,Jeffrey,Skilling,skilling-j,CEO


# Messages

In [4]:
mes_raw = get("raw_message")
mes = mes_raw.rename(columns = {"mid" : "m_id"})
mes = mes.set_index("m_id")
mes = mes[["sender", "date", "subject", "body", "folder"]]
mes = mes.astype({"folder" : "category"})
mes = mes.drop_duplicates()

# Handling emails
add_emails(mes.sender.values)
mes = update_emails(mes, "sender")

#Save
save(mes, "messages")


#Show
get_save(mes_raw, mes)
mes.head(2)

17378 emails added from 252755 emails given (17675 in memory).
87.97% of raw !


,e_id,date,subject,body,folder
m_id,,,,,
52,297,2000-01-21 04:51:00,ENRON HOSTS ANNUAL ANALYST CONFERENCE PROVIDES...,HOUSTON - Enron Corp. hosted its annual equity...,Robert_Badeer_Aug2000Notes FoldersPress releases
53,298,2000-01-24 01:37:00,Over $50 -- You made it happen!,"On Wall Street, people are talking about Enron...",Robert_Badeer_Aug2000Notes FoldersPress releases


# Recipientinfo

In [5]:
rec_raw = get("raw_recipientinfo")
rec = rec_raw.rename(columns = {"rvalue" : "email", "rid" : "r_id", "mid" : "m_id", "rtype" : "type"})
rec = rec.astype({"r_id" : np.uint32, "m_id" : np.uint32, "type" : "category"})
rec = rec.set_index("r_id")
rec = rec[["m_id", "email", "type"]]
rec = rec.drop_duplicates()

# Handling emails
add_emails(rec.email)
rec = update_emails(rec, "email")

#Save
save(rec, "recipients")


get_save(rec_raw, rec)
rec.head()

57889 emails added from 2041168 emails given (75564 in memory).
10.35% of raw !


,m_id,e_id,type
r_id,,,
67,52,17675,TO
68,53,17676,TO
69,54,17677,TO
70,55,17675,TO
71,56,17678,TO


# Reference

In [6]:
ref_raw = get("raw_referenceinfo")
ref = ref_raw.rename(columns = {"rfid" : "rf_id", "mid" : "m_id", "reference" : "content"})
ref = ref.astype({"rf_id" : "uint16", "m_id" : "uint32"})
ref = ref.set_index("rf_id")
ref = ref.drop_duplicates()

#Save
save(ref, "references")


get_save(ref_raw, ref)
ref.head()

99.78% of raw !


,m_id,content
rf_id,,
2,79,"> From: Le Vine, Debi> Sent: Thursday, August ..."
3,99,"> From: Golden, Mark> Sent: Thursday, August 2..."
8,533,"From: Sole, JeanneSent: Friday, August 18, 200..."
13,842,"From: \tGrigsby, Mike Sent:\tThursday, March ..."
14,845,"From: Monaco, John [EM] [mailto:john.monaco@ci..."


# Emails

In [7]:
#Save
save(emails, "emails")